## Before you start:
**Tools → Settings → Editor → completions / suggestions / linting → disable**

## Task 1

Write 2 implementations of the `increment()` function that **increments the global variable `counter` by 1**:

**with and without** (!) Python syntactic sugar. You may not change the function signature.

**1.1: with Python syntactic sugar**

In [ ]:
counter = 0

def increment():
  # your code here
  global counter
  counter += 1

increment()
increment()
assert counter == 2, 'try again'
print(f'{counter=} -- great!')

counter=2 -- great!


**1.2: without Python syntactic sugar**

In [ ]:
counter = 0

def increment():
  # your code here
  global counter
  counter = counter + 1

increment()
increment()
assert counter == 2, 'try again'
print(f'{counter=} -- great!')

counter=2 -- great!


## Task 2

Import **only the `sqrt` function** from the `math` module and execute sqrt(169).  
You may not execute `import math`.

Provide 2 solutions.

...

In [21]:
# your code here
#solution 1

from math import sqrt

result = sqrt(169)
assert result == 13
print('result', result)

result 13.0


In [22]:
#solution 2
def compute():
    from math import sqrt
    return sqrt(169)

result = compute()
assert result == 13
print('result', result)

result 13.0


## Task 3

Dynamic import and reload.

1. Create a module `mod.py`:

2. Import it and print `msg`

3. Change `msg` to `B` in the file

4. Without restarting the notebook session, print the new value of `msg`

In [71]:
import sys
import importlib

# Step 1: create mod.py
with open("mod.py", "w") as f:
    f.write('msg = "A"')

# Make sure don't use an old cached 'mod'
if "mod" in sys.modules:
    del sys.modules["mod"]
importlib.invalidate_caches()

# Step 2: import and print
import mod
print("Initial:", mod.msg)   # should print: A

# Step 3: modify file
with open("mod.py", "w") as f:
    f.write('msg = "B"  # changed to B')

# Step 4: reload
importlib.invalidate_caches()
mod = importlib.reload(mod)
print("After reload:", mod.msg)   # should print: B

Initial: A
After reload: B


## Task 4

You have a directory `pkg`:

In [105]:
!mkdir -p pkg

In [107]:
%%writefile pkg/m1.py
pi = 3.1415_92_65
_e = 2.7
__i = -1

Overwriting pkg/m1.py


```
pkg/
└── m1.py
```

Below are cells for your code; the task follows

### **First way**: via a module from the CPython standard library

In [108]:
%%writefile pkg/__init__.py
import importlib as _importlib

# Import the submodule pkg.m1 as a module object
_m1 = _importlib.import_module(__name__ + ".m1")

# Copy everything from m1 except *symmetric* dunders like __name__, __file__, etc.
# This will include: pi, _e, __i
for _name, _value in vars(_m1).items():
    if not (_name.startswith("__") and _name.endswith("__")):
        globals()[_name] = _value

# Build __all__:
# - all names that do NOT start with a single underscore  -> gives 'pi'
# - plus special "hidden" double-underscore names like '__i'
_hidden_double = [
    name for name in vars(_m1)
    if name.startswith("__") and not name.endswith("__")
]

__all__ = [
    name for name in globals()
    if not name.startswith("_")
] + _hidden_double

# Clean up helper names
del _importlib, _m1, _name, _value, _hidden_double


Overwriting pkg/__init__.py


In [110]:
from pkg import *

print("pi =", pi)



pi = 3.14159265


### **Second way**: in one line without extra modules

In [111]:
# your code here
%%writefile pkg/__init__.py
from . import m1 as _m1; globals().update({k: getattr(_m1, k) for k in dir(_m1) if not k.startswith("_")})


Overwriting pkg/__init__.py


In [112]:
from pkg import *
print(pi)          # should again print: 3.14159265


3.14159265


### Task text:
You may not re-create the values `pi`, `_e`, `__i` or use them directly in the import.  
You must change the structure of the `pkg` package / contents of its modules so that the following code runs correctly:

In [103]:
from pkg import *
pi

3.14159265

**Important!**  
Whenever you update any data in the project directory, you must reload the ipynb session:  
`Runtime --> Restart Session` and re-run the necessary task cells,  
otherwise the results may be incorrect for you.

## Task 5

After correctly solving **Task 4**, you need to:
- modify `pkg`
- complete the code below

so that you can reach `__i`.  
You may not re-create the values `pi`, `_e`, `__i` or use them directly in the import.  
If you restart the session, you must re-run the cells for **Task 4** before solving **Task 5**.

In [121]:
from pkg import *

print("__i =", __i)



__i = -1


In [119]:
from pkg import *
__i

-1

## Task 6

Mutable closure. Why does this code behave unexpectedly? Fix it.

In [ ]:
def create_accumulators():
    accs = []
    for i in range(3):
        def accumulator(x):
            accs[i] += x
            return accs[i]
        accs.append(0)
    return accs

acc_list = create_accumulators()
print(acc_list[0](10))  # Expected 10, but an error occurs

TypeError: 'int' object is not callable

So there two errors in the code

1. Inside the loop we define accumulator, but  never append it.
we append 0 instead.

2. The inner function closes over i, which changes in the loop.
After the loop, i == 2, so all accumulators would use index 2.

In [124]:
# fixed code
def create_accumulators():
    accs = [0, 0, 0]   # underlying storage

    def make_acc(i):
        def accumulator(x):
            accs[i] += x
            return accs[i]
        return accumulator

    return [make_acc(i) for i in range(3)]



In [125]:
acc_list = create_accumulators()

print(acc_list)  # 10
print(acc_list)   # 15

print(acc_list)   # 7
print(acc_list)   # 10

print(acc_list)   # 1

[<function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6598b80>, <function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6598ea0>, <function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6f16980>]
[<function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6598b80>, <function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6598ea0>, <function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6f16980>]
[<function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6598b80>, <function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6598ea0>, <function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6f16980>]
[<function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6598b80>, <function create_accumulators.<locals>.make_acc.<locals>.accumulator at 0x7902d6598ea0>, <function create_

## Task 7
When the notebook session is restarted, the solution starts from scratch

### 7.1: Restore the `print` functionality without using `del`

In [129]:
print = 1

In [130]:
# your code here
print = 1
print

1

In [133]:
import builtins
print = builtins.print
print("Print Restored")

Print Restored


### 7.2: Delete the object `print`, then restore its functionality

In [134]:
# your code here
# Break print
print = 123

# 1) Delete the (shadowing) print object
del print

# 2) Restore real print from builtins
import builtins
print = builtins.print

# Test
print("Hello, world!")  # should work again


Hello, world!


## Task 8

Closure with mutable state. Create a counter function that remembers the number of calls across different instances:

In [ ]:
def make_shared_counter():
    # Your code here

c1 = make_shared_counter()
c2 = make_shared_counter()

print(c1())
print(c2())
print(c1())

In [135]:
# Task 8 correct solution
def make_shared_counter():
    # Shared state stored as an attribute on the function itself
    if not hasattr(make_shared_counter, "count"):
        make_shared_counter.count = 0

    def counter():
        make_shared_counter.count += 1
        return make_shared_counter.count

    return counter

c1 = make_shared_counter()
c2 = make_shared_counter()

print(c1())   # 1
print(c2())   # 2
print(c1())   # 3


1
2
3


## Task 9

Complete the code so that the `outer` function returns **a dictionary with three closures**: `add()`, `mul()`, `get()` — all working with the same closed variable `value`.

In [ ]:
def outer(val=0):
    # your code
    pass

obj = outer(10)
obj['add'](5)
obj['mul'](2)
assert obj['get']() == 30

In [144]:
# Task 09 Full solution
def outer(val=0):

    def add(x):
        nonlocal val
        val += x
        return val

    def mul(x):
        nonlocal val
        val *= x
        return val

    def get():
        return val

    return {'add': add, 'mul': mul, 'get': get}

obj = outer(10)
print(obj['add'](5))
print(obj['mul'](2))
assert obj['get']() == 30






15
30


## Task 10

Create a closure that takes a function and returns a new function with result caching (memoization).

*Theoretical note:*

The `memoize` function accepts another function `func` and creates an inner closure with a dictionary `cache` for storing results.

In the example with the `fib` function (Fibonacci numbers), memoization reduces the number of recursive calls from exponential to linear, since each `n` is computed only once.

Thus, memoization saves time at the expense of memory — a classic "time vs. memory" optimization — and is especially useful for functions with expensive computations and repeated inputs.

Algorithm for solution:

- The memoize function accepts another function func and creates an inner closure with a dictionary cache for storing results.

- The inner wrapper function checks whether a result for the given input argument (x) is already in the cache dictionary.

- If it is, the cached result is returned and no recomputation occurs.

- If not, the original function func(x) is called, the result is saved in cache and returned.


In [ ]:
def memoize(func):
    cache = {}
    def wrapper(x):
        # Your code here
    return wrapper

@memoize
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

print(fib(10))  # 55

In [146]:
def memoize(func):
    cache = {}
    def wrapper(x):
        # If result already computed — return from cache
        if x in cache:
            return cache[x]
        # Otherwise compute and store in cache
        result = func(x)
        cache[x] = result
        return result
    return wrapper


@memoize
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)


print(fib(10))  # 55


55
